## Tests for downloading data from NOAA 

### TODO
* autoloop over dates

In [93]:
# Imports
#needed to make web requests
import requests
#store the data we get as a dataframe
import pandas as pd
#convert the response as a strcuctured json
import json
#mathematical operations on lists
import numpy as np
#parse the datetimes we get from NOAA
from datetime import datetime, timedelta

from joblib import delayed, Parallel

import csv
import tqdm
from noaaplotter.utils import dl_noaa_api

In [94]:
#add the access token you got from NOAA
Token = 'LaVQzwUgOBQLBRwoTpOLyRbIKDTHAVVe'

#Long Beach Airport station
#station_id = 'GHCND:USW00026616' # Kotzebue
station_id = 'GHCND:USW00027502' # Barrow
#station_name = 'Kotzebue'
station_name = 'Barrow'
datatypes = ['TMIN', 'TMAX', 'PRCP', 'SNOW']
date_start = '1971-01-01'
date_end = '2021-12-31'

In [95]:
dtypes_string = '&'.join([f'datatypeid={dt}' for dt in datatypes])

#### Prepare requests

In [96]:
# convert datestring to dt
dt_start = datetime.strptime(date_start, '%Y-%m-%d')
dt_end = datetime.strptime(date_end, '%Y-%m-%d')
# calculate number of days
n_days = (dt_end-dt_start).days
# calculate nuber of splits to fit into 1000 lines/rows
split_size = np.floor(1000 / len(datatypes))
# calculate splits
split_range = np.arange(0, n_days, split_size)

### Run data loading

make joblib

In [97]:
%time datasets_list = Parallel(n_jobs=4)(delayed(dl_noaa_api)(i, dtypes_string, station_id, Token, date_start, date_end, split_size) for i in tqdm.tqdm(split_range[:]))

 11%|████████▊                                                                          | 8/75 [00:02<00:23,  2.89it/s]

AttributeError: module 'datetime' has no attribute 'strptime'

In [98]:
df = pd.concat(datasets_list)

#### Pivot table to correct form

In [99]:
df_pivot = df.pivot(index='date', columns='datatype', values='value')

#### adapt  factor 

In [100]:
df_pivot.loc[:, :] /= 10

#### Prepare data export
Option 1: load as object
Option 2: save to csv

#### Reformat columns to target 

"STATION","NAME","DATE","PRCP","SNWD","TAVG","TMAX","TMIN"

In [101]:
df_pivot = df_pivot.reset_index(drop=False)
df_pivot['DATE'] = df_pivot.apply(lambda x: datetime.fromisoformat(x['date']).strftime('%Y-%m-%d'), axis=1)


In [102]:
df_pivot

datatype,date,PRCP,SNOW,TMAX,TMIN,DATE
0,1971-01-01T00:00:00,0.0,0.0,-32.8,-37.2,1971-01-01
1,1971-01-02T00:00:00,0.0,0.0,-30.0,-41.1,1971-01-02
2,1971-01-03T00:00:00,1.5,2.8,-16.7,-30.6,1971-01-03
3,1971-01-04T00:00:00,0.3,0.5,-16.7,-30.0,1971-01-04
4,1971-01-05T00:00:00,0.0,0.0,-23.3,-28.9,1971-01-05
...,...,...,...,...,...,...
18502,2021-08-29T00:00:00,17.0,NaN,9.4,6.1,2021-08-29
18503,2021-08-30T00:00:00,0.3,NaN,8.9,6.7,2021-08-30
18504,2021-08-31T00:00:00,0.5,NaN,11.1,8.3,2021-08-31
18505,2021-09-01T00:00:00,2.3,NaN,13.3,8.9,2021-09-01


In [103]:
dr = pd.DataFrame(pd.date_range(start=date_start, end=date_end), columns=['DATE'])
dr['DATE'] = dr['DATE'].astype(str)



In [104]:
df_merged = pd.concat([df_pivot.set_index('DATE'), dr.set_index('DATE')], join='outer', axis=1).reset_index(drop=False)

In [105]:
df_merged['STATION'] = station_id
df_merged['NAME'] = station_name

df_merged['TAVG'] = None
df_merged['SNWD'] = None

In [106]:
final_cols = ["STATION","NAME","DATE","PRCP","SNWD","TAVG","TMAX","TMIN"]

no index, make all strings 

In [107]:
df_final = df_merged[final_cols]

In [108]:
df_final = df_final.replace({np.nan: None})

In [109]:
df_final.to_csv('./data/tmp.csv', index=False, quoting=csv.QUOTE_ALL)

Fill empty dates